In [ ]:
import matplotlib.pyplot as plt
from multiomicscellsim import Simulator
from multiomicscellsim.config import SimulatorConfig, TissueConfig

from multiomicscellsim.torch_cpm.config import TorchCPMConfig, TorchCPMCellType
from multiomicscellsim.patterns.config import RDPatternLibrary

from IPython.display import HTML

size = 1024
n_cell_types = 2

# TODOs:
# - Add a distirbution across cell_types across a guideline

# Define some cell types

cell_types = [
    TorchCPMCellType(id=1,
                     name="A",
                     background_adhesion=10.0,
                     cells_adhesion=TorchCPMCellType.build_constant_adhesion_vector(x=10.0, this_id=1, n_types=n_cell_types),
                     preferred_volume=int(0.2*size**2),
                     preferred_local_perimeter=4.0,
                     # RDPatternConfig contains f, k, D_a, D_b params)
                     subcellular_pattern=RDPatternLibrary.get_pattern_by_name('Worms')#.add_noise(to=["d_a", "d_b"], var=.01),
    ),
    TorchCPMCellType(id=2,
                     name="B",
                     background_adhesion=10.0,
                     cells_adhesion=TorchCPMCellType.build_constant_adhesion_vector(x=10.0, this_id=2, n_types=n_cell_types),
                     preferred_volume=int(0.2*size**2),
                     preferred_local_perimeter=4.0,
                     subcellular_pattern=RDPatternLibrary.get_pattern_by_name('Fingerprints')#.add_noise(to=["d_a", "d_b"], var=.01),
    )
]

# Define the global simulation parameters

sim_config = SimulatorConfig(
    tissue_config = TissueConfig(
        n_curves=2,
        curve_types="circles",
        cell_number_mean = 5,
        cell_number_std = 3,
        cell_type_probabilities = [[.3, .7], [.7, .3]],
        initial_cell_size=7,
    ),
    cpm_config = TorchCPMConfig(
        size=size,
        frontier_probability=0.8,
        temperature=1.0,
        cell_types=cell_types,
        lambda_volume=10.0,
        lambda_perimeter=10.0,
        max_cpm_steps=1000,
        run_rd_every=2,
        rd_warmup_steps=10000,
        rd_steps=100,
        device="cuda"
    ),
    save_tissue_every=5,
    seed=42,

)



In [ ]:
sim = Simulator(config=sim_config)
tissues = sim.tissue_generator.sample()

In [ ]:
HTML(sim.plot_tissues(tissues))